## Interaktywny Chatbot Phi 3 Mini 4K Instruct z Whisper

### Wprowadzenie:
Interaktywny Chatbot Phi 3 Mini 4K Instruct to narzędzie, które umożliwia użytkownikom interakcję z demonstracją Microsoft Phi 3 Mini 4K Instruct za pomocą tekstu lub dźwięku. Chatbot może być używany do różnych zadań, takich jak tłumaczenie, aktualizacje pogodowe czy zbieranie ogólnych informacji.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Utwórz swój Huggingface Access Token

Utwórz nowy token  
Podaj nową nazwę  
Wybierz uprawnienia do zapisu  
Skopiuj token i zapisz go w bezpiecznym miejscu


Poniższy kod w Pythonie wykonuje dwa główne zadania: importowanie modułu `os` oraz ustawianie zmiennej środowiskowej.

1. Importowanie modułu `os`:
   - Moduł `os` w Pythonie umożliwia interakcję z systemem operacyjnym. Pozwala na wykonywanie różnych zadań związanych z systemem operacyjnym, takich jak dostęp do zmiennych środowiskowych, operacje na plikach i katalogach itp.
   - W tym kodzie moduł `os` jest importowany za pomocą instrukcji `import`. Ta instrukcja udostępnia funkcjonalność modułu `os` do użycia w bieżącym skrypcie Pythona.

2. Ustawianie zmiennej środowiskowej:
   - Zmienna środowiskowa to wartość, do której mogą mieć dostęp programy działające w systemie operacyjnym. Jest to sposób przechowywania ustawień konfiguracyjnych lub innych informacji, które mogą być używane przez wiele programów.
   - W tym kodzie nowa zmienna środowiskowa jest ustawiana za pomocą słownika `os.environ`. Kluczem w tym słowniku jest `'HF_TOKEN'`, a wartość jest przypisywana z zmiennej `HUGGINGFACE_TOKEN`.
   - Zmienna `HUGGINGFACE_TOKEN` jest zdefiniowana tuż nad tym fragmentem kodu i przypisywana jest jej wartość tekstowa `"hf_**************"` za pomocą składni `#@param`. Ta składnia jest często używana w notebookach Jupyter, aby umożliwić wprowadzanie danych przez użytkownika oraz konfigurację parametrów bezpośrednio w interfejsie notebooka.
   - Ustawiając zmienną środowiskową `'HF_TOKEN'`, można uzyskać do niej dostęp w innych częściach programu lub w innych programach działających na tym samym systemie operacyjnym.

Podsumowując, ten kod importuje moduł `os` i ustawia zmienną środowiskową o nazwie `'HF_TOKEN'` z wartością podaną w zmiennej `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ten fragment kodu definiuje funkcję o nazwie clear_output, która służy do czyszczenia wyników w bieżącej komórce w Jupyter Notebook lub IPython. Przyjrzyjmy się kodowi i zrozummy jego działanie:

Funkcja clear_output przyjmuje jeden parametr o nazwie wait, który jest wartością logiczną (boolean). Domyślnie wait jest ustawiony na False. Ten parametr określa, czy funkcja powinna poczekać, aż nowe wyniki będą dostępne do zastąpienia istniejących, zanim je wyczyści.

Sama funkcja służy do czyszczenia wyników w bieżącej komórce. W Jupyter Notebook lub IPython, gdy komórka generuje wyniki, takie jak tekst wyjściowy czy wykresy, są one wyświetlane poniżej komórki. Funkcja clear_output pozwala na usunięcie tych wyników.

Implementacja funkcji nie została podana w fragmencie kodu, co wskazuje na obecność wielokropka (...). Wielokropek reprezentuje miejsce na właściwy kod, który realizuje czyszczenie wyników. Implementacja funkcji może polegać na interakcji z API Jupyter Notebook lub IPython w celu usunięcia istniejących wyników z komórki.

Podsumowując, funkcja ta zapewnia wygodny sposób na czyszczenie wyników w bieżącej komórce w Jupyter Notebook lub IPython, co ułatwia zarządzanie i aktualizowanie wyświetlanych wyników podczas interaktywnych sesji kodowania.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Wykonaj syntezę mowy (TTS) za pomocą usługi Edge TTS. Przejdźmy przez odpowiednie implementacje funkcji krok po kroku:

1. `calculate_rate_string(input_value)`: Ta funkcja przyjmuje wartość wejściową i oblicza ciąg szybkości dla głosu TTS. Wartość wejściowa reprezentuje pożądaną prędkość mowy, gdzie wartość 1 oznacza normalną prędkość. Funkcja oblicza ciąg szybkości, odejmując 1 od wartości wejściowej, mnożąc wynik przez 100, a następnie określając znak na podstawie tego, czy wartość wejściowa jest większa lub równa 1. Funkcja zwraca ciąg szybkości w formacie "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ta funkcja przyjmuje tekst wejściowy i język jako parametry. Dzieli tekst wejściowy na fragmenty na podstawie reguł specyficznych dla języka. W tej implementacji, jeśli język to "English", funkcja dzieli tekst na kropkach ("."), usuwając wszelkie wiodące lub końcowe spacje. Następnie dodaje kropkę do każdego fragmentu i zwraca przefiltrowaną listę fragmentów.

3. `tts_file_name(text)`: Ta funkcja generuje nazwę pliku dla pliku audio TTS na podstawie tekstu wejściowego. Wykonuje kilka transformacji na tekście: usuwa końcową kropkę (jeśli występuje), konwertuje tekst na małe litery, usuwa wiodące i końcowe spacje oraz zastępuje spacje podkreśleniami. Następnie skraca tekst do maksymalnie 25 znaków (jeśli jest dłuższy) lub używa pełnego tekstu, jeśli jest pusty. Na końcu generuje losowy ciąg za pomocą modułu [`uuid`] i łączy go ze skróconym tekstem, aby utworzyć nazwę pliku w formacie "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ta funkcja łączy wiele plików audio w jeden plik audio. Przyjmuje listę ścieżek plików audio oraz ścieżkę wyjściową jako parametry. Funkcja inicjalizuje pusty obiekt `AudioSegment` o nazwie [`merged_audio`]. Następnie iteruje przez każdą ścieżkę pliku audio, ładuje plik audio za pomocą metody `AudioSegment.from_file()` z biblioteki `pydub` i dodaje bieżący plik audio do obiektu [`merged_audio`]. Na końcu eksportuje połączony plik audio do określonej ścieżki wyjściowej w formacie MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ta funkcja wykonuje operację TTS za pomocą usługi Edge TTS. Przyjmuje listę fragmentów tekstu, prędkość mowy, nazwę głosu oraz ścieżkę zapisu jako parametry. Jeśli liczba fragmentów jest większa niż 1, funkcja tworzy katalog do przechowywania indywidualnych plików audio fragmentów. Następnie iteruje przez każdy fragment, konstruuje polecenie Edge TTS za pomocą funkcji `calculate_rate_string()`, nazwy głosu i tekstu fragmentu, a następnie wykonuje polecenie za pomocą funkcji `os.system()`. Jeśli wykonanie polecenia zakończy się sukcesem, dodaje ścieżkę wygenerowanego pliku audio do listy. Po przetworzeniu wszystkich fragmentów, łączy indywidualne pliki audio za pomocą funkcji `merge_audio_files()` i zapisuje połączony plik audio w określonej ścieżce zapisu. Jeśli jest tylko jeden fragment, bezpośrednio generuje polecenie Edge TTS i zapisuje audio w ścieżce zapisu. Na końcu zwraca ścieżkę zapisu wygenerowanego pliku audio.

6. `random_audio_name_generate()`: Ta funkcja generuje losową nazwę pliku audio za pomocą modułu [`uuid`]. Generuje losowy UUID, konwertuje go na ciąg znaków, pobiera pierwsze 8 znaków, dodaje rozszerzenie ".mp3" i zwraca losową nazwę pliku audio.

7. `talk(input_text)`: Ta funkcja jest głównym punktem wejścia do wykonania operacji TTS. Przyjmuje tekst wejściowy jako parametr. Najpierw sprawdza długość tekstu wejściowego, aby określić, czy jest to długie zdanie (większe lub równe 600 znaków). Na podstawie długości i wartości zmiennej `translate_text_flag` określa język i generuje listę fragmentów tekstu za pomocą funkcji `make_chunks()`. Następnie generuje ścieżkę zapisu dla pliku audio za pomocą funkcji `random_audio_name_generate()`. Na końcu wywołuje funkcję `edge_free_tts()`, aby wykonać operację TTS, i zwraca ścieżkę zapisu wygenerowanego pliku audio.

Podsumowując, te funkcje współpracują ze sobą, aby podzielić tekst wejściowy na fragmenty, wygenerować nazwę pliku dla pliku audio, wykonać operację TTS za pomocą usługi Edge TTS oraz połączyć indywidualne pliki audio w jeden plik audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementacja dwóch funkcji: convert_to_text i run_text_prompt, oraz deklaracja dwóch klas: str i Audio.

Funkcja convert_to_text przyjmuje jako argument audio_path i transkrybuje dźwięk na tekst za pomocą modelu o nazwie whisper_model. Funkcja najpierw sprawdza, czy flaga gpu jest ustawiona na True. Jeśli tak, whisper_model jest używany z określonymi parametrami, takimi jak word_timestamps=True, fp16=True, language='English' i task='translate'. Jeśli flaga gpu jest ustawiona na False, whisper_model jest używany z fp16=False. Wynikowa transkrypcja jest następnie zapisywana do pliku o nazwie 'scan.txt' i zwracana jako tekst.

Funkcja run_text_prompt przyjmuje jako argumenty message i chat_history. Wykorzystuje funkcję phi_demo do wygenerowania odpowiedzi od chatbota na podstawie podanej wiadomości. Wygenerowana odpowiedź jest następnie przekazywana do funkcji talk, która konwertuje odpowiedź na plik audio i zwraca ścieżkę do tego pliku. Klasa Audio jest używana do wyświetlania i odtwarzania pliku audio. Dźwięk jest wyświetlany za pomocą funkcji display z modułu IPython.display, a obiekt Audio jest tworzony z parametrem autoplay=True, dzięki czemu dźwięk zaczyna się odtwarzać automatycznie. chat_history jest aktualizowany o podaną wiadomość i wygenerowaną odpowiedź, a następnie zwracane są pusty ciąg znaków oraz zaktualizowana historia czatu.

Klasa str to wbudowana klasa w Pythonie, która reprezentuje sekwencję znaków. Udostępnia różne metody do manipulacji i pracy z ciągami znaków, takie jak capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill i inne. Te metody pozwalają na wykonywanie operacji takich jak wyszukiwanie, zastępowanie, formatowanie i manipulowanie ciągami znaków.

Klasa Audio to niestandardowa klasa, która reprezentuje obiekt audio. Jest używana do tworzenia odtwarzacza audio w środowisku Jupyter Notebook. Klasa akceptuje różne parametry, takie jak data, filename, url, embed, rate, autoplay i normalize. Parametr data może być tablicą numpy, listą próbek, ciągiem znaków reprezentującym nazwę pliku lub URL, albo surowymi danymi PCM. Parametr filename służy do określenia lokalnego pliku, z którego mają być załadowane dane audio, a parametr url służy do określenia URL, z którego mają być pobrane dane audio. Parametr embed określa, czy dane audio powinny być osadzone za pomocą URI danych, czy odwoływać się do oryginalnego źródła. Parametr rate określa częstotliwość próbkowania danych audio. Parametr autoplay decyduje, czy dźwięk powinien zacząć się odtwarzać automatycznie. Parametr normalize określa, czy dane audio powinny być normalizowane (przeskalowane) do maksymalnego możliwego zakresu. Klasa Audio udostępnia również metody, takie jak reload do ponownego załadowania danych audio z pliku lub URL, oraz atrybuty, takie jak src_attr, autoplay_attr i element_id_attr do pobierania odpowiednich atrybutów dla elementu audio w HTML.

Podsumowując, te funkcje i klasy są używane do transkrypcji dźwięku na tekst, generowania odpowiedzi audio od chatbota oraz wyświetlania i odtwarzania dźwięku w środowisku Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
